<a href="https://colab.research.google.com/github/mrkevinlin/MLmed/blob/master/MLmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade -q gspread

In [4]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

KeyboardInterrupt: ignored

In [0]:
from __future__ import print_function

import math
from google.colab import files
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

In [0]:
# Open Google Sheet of patient data and get column titles
patient_dataframe = gc.open('patient_data.csv').sheet1
column_names = patient_dataframe.row_values(1)

# Process Google Sheet into a pandas dataframe of all numeric values
patient_dataframe = pd.DataFrame.from_records(
    patient_dataframe.get_all_values(),
    columns=column_names)
patient_dataframe = patient_dataframe.iloc[1:]
patient_dataframe = patient_dataframe.apply(pd.to_numeric, errors='coerce')
patient_dataframe = patient_dataframe.reindex(np.random.permutation(patient_dataframe.index))



"""
Attempt to clean up dataframe with usable features
"""
# Print the number of valid results in each column
def count_column_results(df):
  for col in df.columns:
    length = (df.dropna(subset=[col]))[col].count()
    print(col + ": %d" %length)
    
# Remove all rows with any empty cells (14 results left)
def clean_up_all(df):
  df.replace('', np.nan, inplace=True)
  df.dropna(inplace=True)
  return df

# Remove all rows with values missing in the specified columns
def clean_up_by_missing_columns(df, column):
  for col in column:
    df.dropna(subset=[col], inplace=True)
  return df

# Remove any other columns that do not have at least n% the amount of data points as target label
def clean_up_by_target(df, target, n):
  #for col in df.columns:
   # if (df[col].count() < (df[target].count()*n)):
   #   df.drop(col, axis=1, inplace=True)
  return df

target_feature = (['entry_6mwt'])
edited_patient_dataframe = clean_up_by_missing_columns(patient_dataframe, target_feature)
edited_patient_dataframe = clean_up_by_target(edited_patient_dataframe, target_feature, 0.9)
clean_up_by_target(edited_patient_dataframe, target_feature, n=0.9)
#count_column_results(edited_patient_dataframe)

In [0]:
# Preprocess the patient dataframe and divide into training, validation, and test sets
edited_patient_dataframe.describe()